# Scanning Freesolv for Heterocycles

We are going to import the ring_mols dict and only look at 1 an 2 ring systems.  Then we will make another dict where we separate by the number of members in the ring...

After this, each entry in the dict will be checked to make sure that they have the same number of heavy atoms.  perturbations will be conducted on systems where (among the ring members) there is a different neighborhood list.

In [1]:
# get the freesolv database
import requests
from openeye import oechem
from perses.utils.openeye import createOEMolFromSMILES, createOEMolFromIUPAC
import tqdm

In [2]:
mol = createOEMolFromIUPAC('naphthalene')

In [4]:
oechem.OEFindRingAtomsAndBonds(mol)

In [5]:
nrings, parts = oechem.OEDetermineRingSystems(mol)

In [9]:
mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, "C(O)(=O)c1cccc2c1[nH]c(C3CCCc4c3cccc4)c2")

nraromsystems, parts = oechem.OEDetermineRingSystems(mol)

(nraromsystems, parts)

(2, [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1])

In [10]:
print("Aliphatic atoms:", end=" ")
for atom in mol.GetAtoms():
    if parts[atom.GetIdx()] == 0:
        print(atom.GetIdx(), end=" ")
print()

print("Number of aromatic ring systems =", nraromsystems)

for ringidx in range(1, nraromsystems + 1):
    print(ringidx, ". aromatic ring system:", end=" ")
    for atom in mol.GetAtoms():
        if parts[atom.GetIdx()] == ringidx:
            print(atom.GetIdx(), end=" ")
    print()

Aliphatic atoms: 0 1 2 
Number of aromatic ring systems = 2
1 . aromatic ring system: 3 4 5 6 7 8 9 10 21 
2 . aromatic ring system: 11 12 13 14 15 16 17 18 19 20 


In [16]:
mol = createOEMolFromIUPAC('naphthalene')

In [24]:
for atom in mol.GetAtoms():
    if atom.IsInRing():
        print('yes')
    size = oechem.OEAtomGetSmallestRingSize(atom)
    if size == 0:
        print(atom.GetIdx(), "acyclic")
    else:
        print(atom.GetIdx(), "smallest ring size=", size)

yes
0 smallest ring size= 6
yes
1 smallest ring size= 6
yes
2 smallest ring size= 6
yes
3 smallest ring size= 6
yes
4 smallest ring size= 6
yes
5 smallest ring size= 6
yes
6 smallest ring size= 6
yes
7 smallest ring size= 6
yes
8 smallest ring size= 6
yes
9 smallest ring size= 6
10 acyclic
11 acyclic
12 acyclic
13 acyclic
14 acyclic
15 acyclic
16 acyclic
17 acyclic


In [ ]:
def check_for_same_ring_structure(molA, molB):
    """
    -argument: oemolA, oemolB
    -return: bool, True if the architecture is the same (regardless of the atom)
    """
        
    #now the rigorous check to make sure that for each ring, the size/smallest ring sizes are the same
    sizeA = sorted([oechem.OEAtomGetSmallestRingSize(atom) for atom in molA.GetAtoms() if oechem.OEAtomGetSmallestRingSize(atom) != 0])
    sizeB = sorted([oechem.OEAtomGetSmallestRingSize(atom) for atom in molB.GetAtoms() if oechem.OEAtomGetSmallestRingSize(atom) != 0])
    
    if sizeA != sizeB:
        return False

    
    atomic_numsA = sorted([atom.GetAtomicNum() for atom in molA.GetAtoms() if atom.IsInRing()])
    atomic_numsB = sorted([atom.GetAtomicNum() for atom in molB.GetAtoms() if atom.IsInRing()])
    if atomic_numsA != atomic_numsB and len(atomic_numsA) == len(atomic_numsB):
        return True
    else:
        return False
        
        
    
    

In [ ]:
def scan_for_heterocycles(combinations):
    """
    -argument: list of smiles tuples of potential matches
    -return: list of smiles that have the same ring system but differ by 1 or more constituent element
    """
    molA, molB = oechem.OEGraphMol(), oechem.OEGraphMol() 

    for idx in tqdm.trange(combinations):
        smileA, smileB = combinations[idx][0], combinations[idx][1]
        oechem.OESmilesToMol(molA, smileA)
        oechem.OESmilesToMol(molB, smileB)
        
        if check_for_same_ring_structure(molA, molB):
            #if this passes, we can only append the combination if the cycle members don't have the same sorted list of atomic numbers
            
        
        